Reading in the data

In [1]:
import git
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.experimental import enable_halving_search_cv # Needed for HalvingGridSearchCV, which is experimental
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
# Add more imports in this block later. There will need to be several "from sklearn.whatever import something" lines

In [2]:
repo = git.Repo('.', search_parent_directories = True)
root = repo.working_tree_dir

# The sample id and the log-transformed gene expression values.
half_data_1 = pd.read_csv(root + '\\data\\RKNGHStress.csv')
half_data_1 = half_data_1.loc[:, half_data_1.columns.str.startswith(('Sample', 'Log'))]
half_data_1 = half_data_1.rename(columns = {'Sample' : 'sample', 'Log16S' : 'bact', 'Logcbblr' : 'cbblr', 'Log18S' : 'fungi', 'Logphoa' : 'phoa', 'Logurec' : 'urec'})

# The hyperspectral measurements for each sample
half_data_2 = pd.read_csv(root + '\\data\\RKNGHStressPCAPSR.csv')
half_data_2 = half_data_2.rename(columns = {'Unnamed: 0' : 'sample'})

data = half_data_1.join(half_data_2.set_index('sample'), on = 'sample')

TEMP: testing manual construction of models with specific hyperparameters

In [3]:
X = data.drop(['sample', 'bact', 'cbblr', 'fungi', 'phoa', 'urec'], axis = 1)
# NOTE: when doing phoa, there are a couple of samples (3 and 30) that have no data recorded, so we'll need to remove NAs there. But those observations still have data for the other genes.
bact = data[['bact']]

In [4]:
# Note: do NOT scale X and y before splitting, since that is a data leak. Instead, use the pipeline to scale both Xs and the y training, and manually scale the y testing for custom scoring like RMSE.
X_train, X_test, bact_train, bact_test = train_test_split(X.to_numpy(), bact.to_numpy(), train_size = 0.8, random_state = 0)
bact_train = scale(bact_train.ravel())
bact_test = scale(bact_test.ravel())

# For the sake of robustness, maybe should repeat this a few times, with different random states (still manually set for sake of reproducibility) e.g., 0, 1, ... , 4
cv_0 = KFold(n_splits = 5, shuffle = True, random_state = 0)

# n_jobs will need to be adjusted later when running on SCINet (high performance computing clusters). -1 uses all available cores, which might cause a bit of thrashing, but good enough for now.
pipeline = make_pipeline(StandardScaler(), ElasticNetCV(alphas = [0.001389495], l1_ratio = 0.4285714, cv = cv_0, selection = 'random', max_iter = 10000, n_jobs = -1))
pipeline.fit(X_train, bact_train)
pipeline.score(X_test, bact_test)

0.44820667086239085

This looks weird. None of the folds are converging (true for 10 folds, 5 folds, or even just 1 fold with basic ElasticNet) but changing selection to 'random' and max_iter to 10k allowed convergence. 

The score is pretty low. But what metric is the score? If it's not RMSE it's not in the ballpark of the R version. (UPDATE: It's R^2, so it's a bunch of pretty bad scores, actually.)

It looks like ElasticNetCV, and ElasticNet for that matter, don't allow changing the scoring or tuning metric, or at least it's not obvious how. Can the models be evaluated on RMSE just by calling the RMSE function on the preds and targets? Also, does it even make sense to change the tuning metric for this algorithm? (Idk, you probably COULD, but minimizing the sum of squared residuals is good enough.) And what tuning metric does the tidymodels implementation use?

This all might be caused by using the hyperparameter optima found in the R code, but that had a differet train/test split. So it's not optimal here, but it's still probably pretty good, especially since in the analysis of part 1's hyperparameters, there wasn't much variation among the elastic net models' penalties (all tending to be very close to 0) or mixtures (a bit more spread but similar).

What happens if some of its own tuning were allowed?

In [5]:
# print('Preds on X_test')
# print(pipeline.predict(X_test))
# print()
# print('Scaled bact_test')
# print(scale(bact_test))
# print()

# print('MSE')
# mse = mean_squared_error(scale(bact_test), pipeline.predict(X_test))
# print(mse)

print('RMSE')
rmse = root_mean_squared_error(scale(bact_test), pipeline.predict(X_test))
print(rmse)
print()

# print('MSE, no scaling bact_test')
# mse1 = mean_squared_error(bact_test, pipeline.predict(X_test))
# print(mse1)
# print()

# print('RMSE, no scaling bact_test')
# rmse1 = root_mean_squared_error(bact_test, pipeline.predict(X_test))
# print(rmse1)
# print()

RMSE
0.7428279270043695



Uh oh, RMSE of 0.83 is really bad. It doesn't even come close to the R models. What happened?

Looking at the results, it looks like the scaling isn't happening for some reason, or at least not for the predictions. They're all in the 9 range instead of 0 range. Recalculating RMSE after removing scaling on bact_test gave more reasonable results.

But the problem is, we want to be able to compare models among different targets using a common scale, so normalization has to be done with respect to other targets (but NOT the entire dataset for each column since that's data leakage). This should be done before training. But how can this be implemented? (UPDATE: Fixed by manually scaling bact_train and bact_test, separately. But still doesn't solve the issue of getting much higher RMSE than expected.)

Decided to go back and change the cross validation size to 5 instead of 10 in light of the relatively small dataset here.

In [6]:
# Testing how RMSE changes (hopefully improves!) with hyperparameter tuning

# mix_space = np.linspace(0, 1, 8)
# reg_space = np.logspace(-5, 5, 8)

# elastcv = ElasticNetCV(alphas = reg_space, l1_ratio = mix_space, cv = cv_0, selection = 'random', max_iter = 10000, n_jobs = -1, random_state = 0, positive = True)
# estimators_cv = [('scaler', StandardScaler()), ('elastic_net', elastcv)]
# pipeline_hp = Pipeline(estimators_cv, memory = root + '\\cache')
# pipeline_hp.fit(X_train, bact_train)
# pipeline_hp.score(X_test, bact_test)

In [7]:
# print('RMSE')
# # bact_test has already been scaled above
# rmse_hp = root_mean_squared_error(bact_test, pipeline_hp.predict(X_test))
# print(rmse_hp)
# print()

# print(pipeline_hp['elastic_net'].get_params())
# print()


This sucks. The results are even worse than before. I must have done something wrong here, but I haven't figured out what yet. Also I'm trying to figure out what the hyperparameters were that it ended up on. Maybe it's better to just do ElasticNet and the parameter search separately in the pipeline.

In [8]:
# Trying to get a sense of the spread of the data
print('X_train, mean:', np.mean(scale(X_train), axis=0))
print('X_train, std:', np.std(scale(X_train), axis=0))
print('X_test, mean:', np.mean(scale(X_test), axis=0))
print('X_test, std:', np.std(scale(X_test), axis=0))

X_train, mean: [ 2.64987194e-16  6.98253475e-18 -1.82942410e-16 ...  4.69994414e-15
  5.29555435e-15  1.12698111e-15]
X_train, std: [1. 1. 1. ... 1. 1. 1.]
X_test, mean: [-8.32667268e-18  0.00000000e+00  3.99680289e-16 ... -3.17627868e-15
 -7.89299182e-16 -4.98863104e-16]
X_test, std: [1. 1. 1. ... 1. 1. 1.]


In [9]:
print('bact_train, mean:', np.mean(bact_train))
print('bact_train, std:', np.std(bact_train))
print('bact_test, mean:', np.mean(bact_test))
print('bact_test, std:', np.std(bact_test))

bact_train, mean: -4.2034859171342405e-16
bact_train, std: 0.9999999999999998
bact_test, mean: -1.2934098236883074e-15
bact_test, std: 1.0


In [10]:
print(type([1,2,3]))
print(type(list((1,2,3))))
print(type(list(np.arange(3))))

<class 'list'>
<class 'list'>
<class 'list'>


In [34]:
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("elastic_net", ElasticNet(warm_start=True, positive=True, random_state=0, selection="random"))
    ],
    memory = root+'\\cache'
)

REGULARIZATION = np.logspace(-5, 5, 8) # If this doesn't work, may have to enclose the RHS in list()
MIXTURE = np.linspace(0, 1, 8)
param_grid = [
    {
        "elastic_net__alpha": REGULARIZATION,
        "elastic_net__l1_ratio": MIXTURE
    }
]

grid = HalvingGridSearchCV(estimator=pipe, param_grid=param_grid, factor=2, n_jobs=-1, cv=5, verbose=2)
grid.fit(X_train, bact_train)

n_iterations: 5
n_required_iterations: 7
n_possible_iterations: 5
min_resources_: 10
max_resources_: 318
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 64
n_resources: 10
Fitting 5 folds for each of 64 candidates, totalling 320 fits


C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


----------
iter: 1
n_candidates: 32
n_resources: 20
Fitting 5 folds for each of 32 candidates, totalling 160 fits


C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan -8.55246465 -3.34014611
 -3.34014611 -3.34014611 -

----------
iter: 2
n_candidates: 16
n_resources: 40
Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan -8.55246465 -3.34014611
 -3.34014611 -3.34014611 -

----------
iter: 3
n_candidates: 8
n_resources: 80
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 4
n_candidates: 4
n_resources: 160
Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan -8.55246465 -3.34014611
 -3.34014611 -3.34014611 -

HalvingGridSearchCV(estimator=Pipeline(memory='C:\\Users\\joshua.waldbieser\\OneDrive '
                                              '- '
                                              'USDA\\root_knot_nematode_greenhouse\\paper_2\\DirtSpectra\\cache',
                                       steps=[('scaler', StandardScaler()),
                                              ('elastic_net',
                                               ElasticNet(positive=True,
                                                          random_state=0,
                                                          selection='random',
                                                          warm_start=True))]),
                    factor=2, n_jobs=-1,
                    param_grid=[{'elastic_net__alpha': array([1.00000000e-05, 2.68269580e-04, 7.19685673e-03, 1.93069773e-01,
       5.17947468e+00, 1.38949549e+02, 3.72759372e+03, 1.00000000e+05]),
                                 'elastic_net__l1_ratio': array([0.        , 0.14285714, 0.28571429, 0.42857143, 0.57142857,
       0.71428571, 0.85714286, 1.        ])}],
                    verbose=2)

In [12]:
print(grid.score(X_test, bact_test))
print('RMSE:', root_mean_squared_error(bact_test, grid.predict(X_test)))

0.0
RMSE: 1.0


Still need to do a postmortem here, but at least the halving search converged quickly...?

I think the issue might be up top when reading in the data. Maybe I made a mistake reading it in, or scaling it, or something I haven't thought of yet. If the issue is with scaling it and the pipeline, maybe try manually scaling and manually using an estimator to confirm.

In [35]:
# Checklist:
# data doesn't have any obvious problems that I can see, except for the missing phoa data for two samples
# bact looks okay
# X looks okay
# X_train and bact_train have corresponding shapes
# Same for _test
# No obvious problems with X_train, bact_train, X_test, or bact_test

[[0.     0.     0.     ... 6.1551 6.1173 6.0873]
 [0.     0.     0.0076 ... 6.0783 6.0551 6.0367]
 [0.256  0.4098 0.5861 ... 5.9398 5.9335 5.9349]
 ...
 [0.0279 0.     0.     ... 5.1607 5.1592 5.158 ]
 [0.     0.1037 0.2165 ... 5.5664 5.5366 5.5023]
 [0.8228 0.9239 0.9861 ... 5.5042 5.5665 5.6117]]

[ 0.34262462  1.14764766  1.25789747  0.36053373  0.297648    0.72490281
 -1.348513    0.64727151  0.76242858 -1.13160151 -1.31539917 -0.97205135
  0.71045427  1.0259603  -1.22188555 -1.348513    0.08538047 -0.42458962
  1.30275516 -1.38355277 -1.55781191  1.31571512  1.31571512  0.36053373
 -1.24485453 -0.5656158  -0.99277195 -0.42458962 -1.13160151  1.14764766
 -1.07963292 -1.19355052  1.32073227 -1.22188555 -1.10715692  1.31571512
  0.4738625  -0.78153368 -0.78153368  0.86034342  1.25790857  0.42132533
 -1.348513   -1.47727873  0.03088268  1.10220764 -1.11571683 -1.02786767
  0.97009603  0.61580049 -0.97205135  0.61580049 -1.22188555  0.72490281
  0.76242858  1.20927013  0.49637395 -1.10

Now the halving grid search is giving a bunch of nan values during training, and clean R2=0.0 and RMSE=1.0 vals. How did this get even worse??? I only manually deleted the two outlier lines from the csv. Next step: try switching to Grid search. If that doesn't work, try manually scaling and then manually fitting elastic net.